In [75]:
# Load libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np

In [112]:
# load dataset
train_data = pd.read_csv("train.csv")
# test_data = pd.read_csv("test.csv")
# test_data = test_data.drop("index",axis=1)
# data = pd.concat([train_data,test_data],sort=False)

In [77]:
data.shape

(258210, 14)

In [102]:
data["price"].max()

2500.0

In [78]:
len(data.columns)

14

In [79]:
cols = list(data.columns)

In [80]:
cols.remove("price")

In [81]:
cols.remove("id")

In [82]:
cols

['country',
 'description',
 'designation',
 'points',
 'province',
 'region_1',
 'region_2',
 'taster_name',
 'taster_twitter_handle',
 'title',
 'variety',
 'winery']

In [83]:
len(data["country"].unique())

49

In [84]:
pd.get_dummies(data["country"])

,Albania,Argentina,Armenia,Australia,Austria,Bosnia and Herzegovina,Brazil,Bulgaria,Canada,Chile,...,Slovenia,South Africa,South Korea,Spain,Switzerland,Turkey,US,US-France,Ukraine,Uruguay
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
d = pd.get_dummies(train_data[["country","region_1","region_2","province","taster_name","variety"]])
d["points"]= train_data["points"]
d["id"] = train_data['id']

In [94]:
X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [114]:
X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test

In [115]:
X_train = X_train.drop(["id"],axis=1)

In [116]:
%%time
# Create Decision Tree classifer object
clf = LinearRegression(n_jobs=10)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

CPU times: user 2min 9s, sys: 445 ms, total: 2min 10s
Wall time: 2min 10s


In [117]:
#Predict the response for test dataset
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

In [118]:
y_pred = clf.predict(X_test)

In [122]:
y_pred[y_pred>2500] = 50
y_pred[y_pred<0]=1

In [123]:
print(y_test[:10],y_pred[:10])

10709      9.0
27315     28.0
129593    38.0
23186     17.0
61957     18.0
128582    22.0
83535     20.0
65190     13.0
100308    28.0
24450     16.0
Name: price, dtype: float64 [14 31 38 96 16 22 21 14 20  0]


In [108]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

In [109]:
result["price"]=result["price"].apply(int)

In [110]:
result.to_csv('group_6_output.csv',index=False)

In [124]:
y_pred = np.int64(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.03304761904761905


In [125]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 30.434130902812452
